In [1]:
import DALI as dali_code
from joblib import Parallel, delayed
import torchaudio
from DALI.extra import audio_from_url
import os

from copy import deepcopy
from tqdm import tqdm
import json

In [3]:
dali_data_path = os.path.abspath("data/DALI_V1")
path_audio = os.path.abspath("data/DALI_v1/songs/")
path_audio_segments "data/DALI_V1/song_segments/"
path_segment_labels = "data/DALI_V1/labels/"

os.makedirs(pathaudio, exist_ok=True)
os.makedirs(path_audio_segments, exist_ok=True)
os.makedirs(path_segment_labels, exist_ok=True)

In [4]:
dali_data = dali_code.get_the_DALI_dataset(dali_data_path, skip=[], keep=[])

ERROR: not files with extension .gz


In [ ]:
dali_info = dali_code.get_info(dali_data_path + 'info/DALI_DATA_INFO.gz')

In [ ]:
# dali_data.keys()

In [ ]:
entry = dali_data['001940b614eb43f4a0c826d49a67d66d']

In [ ]:
gt = dali_code.utilities.read_gzip(f"{dali_data_path}001940b614eb43f4a0c826d49a67d66d.gz")

In [ ]:
# gt.annotations["annot"]["words"]

In [ ]:
# gt.annotations

In [ ]:
# dali_info

In [ ]:
def get_audio_from_url(i):
    path_output = path_audio + i[0] + ".mp3"
    if os.path.exists(path_output):
        return
    try:
        audio_from_url(i[-2], i[0], path_audio, errors)
    except:
        print("Failed ", i[0])

## Download youtube

In [ ]:
errors = []
Parallel(n_jobs=1024, verbose=1)(delayed(get_audio_from_url)(i)
    for i in dali_info[1:])

In [ ]:
def process_info(info):
    ann_path = f"{dali_data_path}{info[0]}.gz"
    audio_path = f"{path_audio}{info[0]}.mp3"
    
    if not os.path.exists(audio_path):
        return
    
    audio, sample_rate = torchaudio.load(audio_path)
    gt = dali_code.utilities.read_gzip(ann_path)
    
    anns = gt.annotations["annot"]["words"]
    
    zalo_anns = []
    
    segment = {"s": 0, "e": 0, "l": []}
    current_end = 30
    
    while anns:
        if anns[0]["time"][1] < current_end:
            ann = anns.pop(0)
            segment["l"].append({"s": ann["time"][0], "e": ann["time"][1], "d": ann["text"]})
        else:
            current_end = anns[0]["time"][0] + 30
            zalo_anns.append(deepcopy(segment))
            segment["l"] = []
    
    
    for i, segment in enumerate(zalo_anns):
        if len(segment["l"]) == 0:
            continue
        if i != 0:
            segment["s"] = segment["l"][0]["s"]
        segment["e"] = segment["l"][-1]["e"]
        # dump segments
        start = int(segment["s"] * sample_rate)
        end = int(segment["e"] * sample_rate)
        audio_segment = audio[:, start : end]
        audio_segment_path = os.path.join(path_audio_segments,
                                          info[0] + f"_{str(i)}.wav")
        torchaudio.save(audio_segment_path, audio_segment, sample_rate)
        
        # make it to 30s
        for word in segment["l"]:
            word["s"] = (word["s"] - segment["s"]) * 1000
            word["e"] = (word["e"] - segment["s"]) * 1000
        segment["e"] = (segment["e"] - segment["s"]) * 1000
        segment["s"] = 0
        json_path = os.path.join(path_segment_labels, info[0] + f"_{str(i)}.json")
        json.dump([segment], open(json_path, "w"), indent=4)

## Process jsons

In [ ]:
Parallel(n_jobs=32, verbose=1)(delayed(process_info)(info) for info in dali_info[1:])

## Remove audios

In [ ]:
from IPython.display import Audio 
from IPython.core.display import display
def beep(path):
    display(Audio(path, autoplay=True))

In [ ]:
from dataset import get_audio_label_paths, load_wave

In [ ]:
audios, labels = get_audio_label_paths("data/DALI_V1/song_segments/", "data/DALI_V1/labels/")

In [ ]:
removed = []
def load_(path):
    audio, sr = torchaudio.load(path)
    if audio.shape[1] == 0:
        removed.append(path)

In [ ]:
"data/DALI_V1/song_segments/ee222691643f40abb4f7f49d6e990766_5.wav" in audios

In [ ]:
_ = Parallel(n_jobs=32, verbose=1)(delayed(load_)(p) for p in audios)

In [ ]:
for p in removed:
    os.remove(p)

In [ ]:
import numpy as np
import json

In [ ]:
idx = np.random.randint(len(labels))
audio = audios[idx]
label = labels[idx]

In [ ]:
json.load(open(label))

In [ ]:
beep(audio)